# More 입력과 출력

## 파일과 스트림

In [1]:
:t getContents

-- 필요할 때 터미널에서 입력을 읽는다.
-- 언제 읽는지, 얼마나 읽는지 등의 세부사항을 대신 처리한다.

getContents :: IO String

In [4]:
import qualified Data.Char as Char

main = do
  contents <- getContents
  -- 이 라인은, 문자열이 언젠가 contents에 쌓일것이라고 약속하는 것에 가까움
  -- 이 작업은 EOF 문자가 나올때까지 반복된다.
  putStr $ map Char.toUpper contents
  -- contents에 인자(Char)가 계속 쌓이고 쌓일 때마다 toUpper 함수가 동작하여 putStr에 전달

In [8]:
shortLinesOnly :: String -> String
shortLinesOnly = unlines . filter (\line -> length line < 10) . lines

main = interact shortLinesOnly
:t interact
-- 입력(String)을 받은 후 출력(String 하는 함수)

interact :: (String -> String) -> IO ()

In [ ]:
isPal :: String -> Bool
isPal xs = xs == reverse xs

responsePalindromes :: String -> String
responsePalindromes =
    unlines .
    map (\xs -> if isPal xs then "palindrome" else "not a palindrome") .
    lines
    
main = interact responsePalindromes
main

not a palindrome
palindrome
not a palindrome
Press Ctrl-C again to quit kernel.

In [6]:
import qualified System.IO as IO

:t IO.openFile
-- 경로와 IOMode를 받아 파일의 핸들을 반환

:t IO.hGetContents
-- 핸들을 통해 내용을 반환함
-- 핸들은 단지 파일의 현재 위치를 가리킬 뿐, 내용은 실제 파일에 있는 것이다. (북마크와 비슷)

:t IO.hClose
-- 핸들을 종료함

IO.openFile :: FilePath -> IOMode -> IO Handle

IO.hGetContents :: Handle -> IO String

IO.hClose :: Handle -> IO ()

In [8]:
import qualified System.IO as IO

:t IO.withFile
-- 파일경로, IOMode, 핸들을 통해 IO작업을 하는 함수를 받아 함수를 실행하고 반환값을 반환
-- 그리고 핸들을 종료해준다! 예외가 발생하더라도!

IO.withFile :: forall r. FilePath -> IOMode -> (Handle -> IO r) -> IO r

In [9]:
import qualified Control.Exception as Exception

:t Exception.bracket
-- 1) 파일 핸들러와 같은 리소스를 얻는 IO작업
-- 2) 리소스를 해제하는 함수 (예외가 발생해도 무저건 호출하는 함수 finally 같은것)
-- 3) 리소스를 받아서 작업하는 함수 (메인 함수이다)
-- 4) 반환

Exception.bracket :: forall a b c. IO a -> (a -> IO b) -> (a -> IO c) -> IO c

In [10]:
import qualified System.IO as IO

:t IO.readFile

IO.readFile :: FilePath -> IO String

In [13]:
import qualified System.IO as IO

main = do
    content <- readFile "./09-input-output-2/girlfriend.txt"
    -- 파일이름으로 쉽게 열고, 닫기까지 해준다.
    putStr content
    
main

Hey! Hey! You! You!   
I don't like your girlfriend!   
No way! No way!   
I think you need a new one!

In [17]:
import qualified System.IO as IO
import qualified Data.Char as Char

main = do 
    contents <- IO.readFile "./09-input-output-2/girlfriend.txt"
    IO.writeFile "./09-input-output-2/girlfriendcaps.txt" (map Char.toUpper contents)
main

In [18]:
import qualified System.IO as IO

:t IO.appendFile
-- 이미 존재하는 파일이 있더라도 덮어쓰지 않고, 뒤에서부터 추가한다.

IO.appendFile :: FilePath -> String -> IO ()

In [3]:
import qualified System.IO as IO

main = do 
    todoItem <- getLine
    appendFile "./09-input-output-2/todo.txt" (todoItem ++ "\n")
    
main

 rudolf


In [ ]:
import qualified System.IO as IO
import qualified System.Directory as Directory
import qualified Data.List as List

main = do
    contents <- readFile "./09-input-output-2/todo.txt"
    let todoTasks = lines contents
        numberedTasks = zipWith (\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStrLn "These are your TODO items:"
    mapM_ putStrLn numberedTasks
    putStrLn "which one do you want to delete?"
    numberString <- getLine
    let number = read numberString
        newTodoItems = unlines $ List.delete (todoTasks !! number) todoTasks
    (tempName, tempHandle) <- IO.openTempFile "." "temp"
    IO.hPutStr tempHandle newTodoItems
    IO.hClose tempHandle
    Directory.removeFile "todo"
    Directory.renameFile tempName "todo.txt"
    
main

These are your TODO items:
0 - rudolf
which one do you want to delete?

In [1]:
import qualified Control.Exception as Exception

:t Exception.bracket
:t Exception.bracketOnError
-- bracket과 달리, 2번째 인자 함수는 에러 발생시에만 호출된다.
-- bracket의 2번째 인자는 finally 처럼 매번 마지막에 작동된다.

Exception.bracket :: forall a b c. IO a -> (a -> IO b) -> (a -> IO c) -> IO c

Exception.bracketOnError :: forall a b c. IO a -> (a -> IO b) -> (a -> IO c) -> IO c

## 커맨드 라인 인자

In [2]:
import qualified System.Environment as Env

:t Env.getArgs
:t Env.getProgName

Env.getArgs :: IO [String]

Env.getProgName :: IO String

## 랜덤

In [4]:
import qualified System.Random as Random

:t Random.random
:t Random.mkStdGen

Random.random :: forall a g. (Random a, RandomGen g) => g -> (a, g)

Random.mkStdGen :: Int -> StdGen

In [10]:
import qualified System.Random as Random

Random.random (Random.mkStdGen 100) :: (Int, Random.StdGen)
Random.random (Random.mkStdGen 100) :: (Int, Random.StdGen)

(-3633736515773289454,693699796 2103410263)

(-3633736515773289454,693699796 2103410263)

In [12]:
import qualified System.Random as Random

Random.random (Random.mkStdGen 949494) :: (Int, Random.StdGen)
Random.random (Random.mkStdGen 949494) :: (Float, Random.StdGen)
Random.random (Random.mkStdGen 949494) :: (Bool, Random.StdGen)
Random.random (Random.mkStdGen 949494) :: (Integer, Random.StdGen)

(6642787099209953655,75809027 2103410263)

(0.7463806,466647808 1655838864)

(False,1485872359 40692)

(6642787099209953655,75809027 2103410263)

In [18]:
import qualified System.Random as Random

threeCoins :: Random.StdGen -> (Bool, Bool, Bool)
threeCoins gen =
    let (firstCoin, newGen) = Random.random gen :: (Bool, Random.StdGen)
        (secondCoin, newGen') = Random.random newGen :: (Bool, Random.StdGen)
        (thirdCoin, newGen'') = Random.random newGen' :: (Bool, Random.StdGen)
    in (firstCoin, secondCoin, thirdCoin)

In [19]:
import qualified System.Random as Random

threeCoins (Random.mkStdGen 21)
threeCoins (Random.mkStdGen 22)
threeCoins (Random.mkStdGen 943)
threeCoins (Random.mkStdGen 944)

(True,True,True)

(True,False,True)

(True,False,True)

(True,True,True)

In [21]:
import qualified System.Random as Random

take 5 $ Random.randoms (Random.mkStdGen 11) :: [Int]
take 5 $ Random.randoms (Random.mkStdGen 11) :: [Bool]
take 5 $ Random.randoms (Random.mkStdGen 11) :: [Float]

[5260538044923710387,4361398698747678847,-8221315287270277529,7278185606566790575,1652507602255180489]

[True,True,True,True,False]

[0.26201087,0.1271351,0.31857032,0.1921351,0.31495118]

In [32]:
import qualified System.Random as Random

randoms' :: (Random.RandomGen g, Random.Random a) => g -> [a]
randoms' gen = let (value, newGen) = Random.random gen :: (a, g)
               in value : randoms' newGen

: 

## thunk, bytestring

thunk는 기본적으로 지연계산이다. 썽크를 이용하여 필요할 때만 계산하는 느긋함을 부린다.

In [1]:
import qualified Data.ByteString.Lazy as B  
import qualified Data.ByteString as S  

B.pack [99,97,110]
B.pack [98..120]


"can"

"bcdefghijklmnopqrstuvwx"

In [2]:
import qualified Data.ByteString.Lazy as B  
import qualified Data.ByteString as S  

by = B.pack [98, 111, 114, 116]
by
B.unpack by

"bort"

[98,111,114,116]

In [3]:
import qualified Data.ByteString.Lazy as B  
import qualified Data.ByteString as S  

B.fromChunks [S.pack [40,41,42], S.pack [43,44,45], S.pack [46,47,48]]  

"()*+,-./0"

In [5]:
import qualified Data.ByteString.Lazy as B  
import qualified Data.ByteString as S  

B.cons 85 $ B.pack [80,81,82,84]  
B.cons' 85 $ B.pack [80,81,82,84]  
foldr B.cons B.empty [50..60]  
foldr B.cons' B.empty [50..60]  

"UPQRT"

"UPQRT"

"23456789:;<"

"23456789:;<"